# MatrixTable

If you've gotten this far, you're probably thinking:
 1. "Can't I do all of this in `pandas` or `R`?" 
 2. "What does this have to do with biology?"

Fear not!  You've learned most of the basic concepts of Hail and now you're ready for the bit that makes it easy to work with genetic data: the [MatrixTable](https://hail.is/docs/devel/hail.MatrixTable.html).

In the last example, the ratings table had two keys: `movie_id` and `user_id`.  The ratings were secretly a movie by user matrix!

Since rating is very sparse, it might be reasonable to store in this way in so-called "coordinate form" where each rating is a row of a table.

However, the per-row overhead becomes untenable when working with large, dense datasets like sequence data!  That's why we built `MatrixTable`, a 2-dimensional generalization of `Table`.

# MatrixTable Anatomy

Remember `Table` had two kinds of fields: globals and row fields.

`MatrixTable` has four kinds of fields:
 - the globals
 - the row fields
 - the column fields
 - the entry fields

Row fields are fields that are stored once per row.  These are like INFO fields in a VCF.

Column fields are stored once per column.  These have no analog in VCF files, but for sequence data, column fields can store sample metadata, phenotypes, etc.

And, of course, entry fields are like the FORMAT fields in a VCF.

# Importing and Reading

Like tables, matrix tables can be *imported* from a variety of formats: VCF, (B)GEN, PLINK, TSV, etc.  Matrix tables can also be *read* from a "native" matrix table format.  Let's read a sample of prepared 1KG data.

In [ ]:
import hail as hl
import matplotlib.pyplot as plt
import seaborn
seaborn.set()
%matplotlib inline

hl.utils.get_1kg('data/')

In [ ]:
mt = hl.read_matrix_table('data/1kg.mt')
mt.describe()

There are a few things to note:
 - There is a single column field `s`.  This is the sample ID from the VCF.  Note, it is the column key.
 - There is a compound row key: `locus` and `alleles`.  
     - `locus` has type `locus<GRCh37>`
     - `alleles` has type `array<str>`
 - GT has type `call`.  That's a genotype call!
 - There is another key type: the partition key. Don't worry about this right now.

Whereas table expressions could be indexed by nothing or indexed by rows, matrix table expression have four options: nothing, indexed by rows, indexed by cols, or indexed by rows and cols (entries).  Let's see some examples.

In [ ]:
mt.s.describe()

In [ ]:
mt.GT.describe()

We belabored the operations on `Table`s because they all have natural analogs (sometimes several) on `MatrixTable`s.  For example:
 - `count` => `count_{rows, cols}` (and `count` which returns both)
 - `filter` => `filter_{rows, cols, entries}`
 - `annotate` => `annotate_{rows, cols, entries}` (and globals for both)
 - same for `select` and `transmute`
 - `group_by` => `group_{rows, cols}_by`
 - `explode` => `expode_{rows, cols}`
 - `aggregate` => `aggregate_{rows, cols, entries}`

There are a few more operations special to `MatrixTable`.  One is that you can get the row fields, as a `Table`, with [rows](https://hail.is/docs/devel/hail.MatrixTable.html#hail.MatrixTable.rows), the column fields with [cols](https://hail.is/docs/devel/hail.MatrixTable.html#hail.MatrixTable.cols).  You can also get the entries as a table, along with the corresponding row and column fields, duplicated for each entry in "coordinate form" with [entries](https://hail.is/docs/devel/hail.MatrixTable.html#hail.MatrixTable.entries).

Let's explore `mt` using these tools.  Let's get the size of the dataset.

In [ ]:
mt.count() # (rows, cols)

Let's look at the first few row keys (variants) and column keys (sample IDs).

In [ ]:
mt.rows().select('locus', 'alleles').show()

In [ ]:
mt.s.show()

Let's investigate the genotypes and the call rate.  Let's look at the first few genotypes:

In [ ]:
mt.GT.show()

All homozygous reference, which is not surprising.  Let's look at the distribution of genotype calls:

In [ ]:
mt.aggregate_entries(hl.agg.counter(mt.GT.n_alt_alleles()))

Let's compute the call rate directly, and then plot the distribution of call rates per vairant.

In [ ]:
mt.aggregate_entries(hl.agg.fraction(hl.is_defined(mt.GT)))

Here's a nice trick: you can use an aggregator inside `annotate_rows` and it will aggregate over columns, that is, summarize the values in the row using the aggregator.  Let's compute and plot call rate per variant.

In [ ]:
mt2 = mt.annotate_rows(call_rate = hl.agg.fraction(hl.is_defined(mt.GT)))
mt2.describe()

In [ ]:
hist_start = 0
hist_end = 1.0
n_bins = 100
bin_size = (hist_end - hist_start) / n_bins

hist = mt2.aggregate_rows(hl.agg.hist(mt2.call_rate, hist_start, hist_end, n_bins))

plt.xlim(0.0, 1.0)
plt.bar(hist.bin_edges[:-1], hist.bin_freq, width=bin_size, label='call_rate')
plt.xlabel('Call Rate')
plt.ylabel('Count')
plt.title('Variant Call Rate Histogram')
plt.legend(loc=2)
plt.show()

# GQ vs DP

Let's plot the distribution of GQ and DP values.

In [ ]:
hist = mt.aggregate_entries(hl.agg.hist(mt.DP, 0, 40, 40))

plt.xlim(0, 40)
plt.xlabel('DP')
plt.ylabel('Count')
plt.title('DP Histogram')
plt.bar(hist.bin_edges[:-1], hist.bin_freq, width=1, label='DP')
plt.legend()
plt.show()

Now, let's do the same thing for GQ.

In [ ]:
hist = mt.aggregate_entries(hl.agg.hist(mt.GQ, 0, 100, 100))

# note to future self, hist should have start and end, and this should just be hl.plot.hist(hist)
plt.xlim(0, 100)
plt.xlabel('GQ')
plt.ylabel('Count')
plt.title('GQ Histogram')
plt.bar(hist.bin_edges[:-1], hist.bin_freq, width=1, label='GQ')
plt.legend()
plt.show()

# Exercise

Whoa!  That's a strange distribution!  There's a huge spike at 100, and the main distribution has the same shape as the DP distribution, but looks like a [Dimetrodon](https://en.wikipedia.org/wiki/Dimetrodon).  Use Hail to figure out what's going on!